# Bank Churn Predection 
- Problem Statement: Finding churn rate of a customer for bank by using deeplearning and ann's

# Import Necessary Libraries 

In [1]:
# Import libraries

# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl

# Loading & Cleaning Dataset

In [2]:
# Load churn dataset

churn_data = pd.read_csv('bank_churn.csv')
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Data shape
num_c=len(churn_data.columns)
num_r = len(churn_data.index) # Dataset consist of 10,000 rows/observations and 14 columns/features
print(f'Number of features/columns: {num_c}\nNumber of observations/rows: {num_r}')

Number of features/columns: 14
Number of observations/rows: 10000


In [4]:
# Check missing values
churn_data.isnull().sum() 

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

**No missing values found in the dataset i.e., data is clean**

In [5]:
# Check duplicate data
churn_data[churn_data.duplicated()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


**No duplicate data has been found in the dataser**

In [6]:
# Dataset information
churn_data.info() # This shows dataset information like column names, null-value, and Data-type of each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


# Feature Engineering

- Transforming raw data into insightful data to improve features to build better models using techniques like, handling missing values, Categorical Encoding, and variable transformation.

In [7]:
# Display first few rows/observations
churn_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
churned = churn_data['Exited'].value_counts()[1]
not_churned = churn_data['Exited'].value_counts()[0]
print(f'Number of customers Exited: {churned}\nNumber of customers stayed: {not_churned}')

Number of customers Exited: 2037
Number of customers stayed: 7963


In [9]:
# Display column names
churn_data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [10]:
# Drop the irrelevant columns
drop_columns = ['RowNumber','CustomerId','Surname']
churn_data = churn_data.drop(drop_columns, axis=1)

In [11]:
# After dropping the irrelevant columns
churn_data.head(8)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1


In [12]:
# Convert gender column category values to numeric values (Female = 0 & Male = 1) using label encoding.
from sklearn.preprocessing import LabelEncoder

lab_gender = LabelEncoder()
churn_data['Gender'] = lab_gender.fit_transform(churn_data['Gender'])

In [13]:
churn_data.head(8)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1


In [14]:
# OneHotEncoding 
from sklearn.preprocessing import OneHotEncoder

geo_ohe = OneHotEncoder()
geo_encode = geo_ohe.fit_transform(churn_data[['Geography']])
geo_encode

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [15]:
geo_col = geo_ohe.get_feature_names_out(['Geography'])
geo_col

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [16]:
encoded_df = pd.DataFrame(geo_encode.toarray(), columns=geo_col)
encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [17]:
# Concat original data frame and geo_df 
transformed_churn_data = pd.concat([churn_data.drop('Geography', axis=1), encoded_df], axis=1)
transformed_churn_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


# Split dataset into dependent and independent features

In [23]:
# Train-test split
X = transformed_churn_data.drop(columns=["Exited"])
y = transformed_churn_data["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:

# Display the shapes of training and testing sets
print(f"Training Data Shape: {X_train.shape}")
print(f"Testing Data Shape: {X_test.shape}")

Training Data Shape: (8000, 12)
Testing Data Shape: (2000, 12)


In [25]:
# Scaling the features using StandardScaler
scaled = StandardScaler()
X_train = scaled.fit_transform(X_train)
X_test = scaled.transform(X_test)

# Creating PickleFiles to deployment

In [26]:
# 1 Creating pickle file of gender encoder
with open('lab_gender.pkl','wb') as file:
    pkl.dump(lab_gender, file)

# 2 Creating pickle file of geography encoder
with open('geo_ohe.pkl','wb') as file:
    pkl.dump(geo_ohe, file)

# 3 Creating pickle file of scaled features
with open('scaled.pkl', 'wb') as file:
    pkl.dump(scaled,file)